<h1>Data preprocessing!</h1>

In [25]:
# Usual import
from pprint import pprint
import numpy as np
import pandas as pd
from tqdm import tqdm
import string

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.utils import simple_preprocess


#For language detection
from langdetect import detect

# spaCy for tokenization, stop word removal and  lemmatization
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
#!python -m spacy download en_core_web_lg


In [38]:
# Create our list of punctuation marks
punctuations = string.punctuation
stopwords = list(STOP_WORDS)
# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
parser = English()

<h2>Load Data</h2>

In [3]:
#data = pd.read_csv("C:/Users/Iro Sfoungari/Desktop/THESIS DATA/CORD-19/metadata.csv")
data = pd.read_csv("C:/Users/Iro Sfoungari/Desktop/THESIS DATA/CORD-19/1000_metadata.csv")
data.head(2)

,Unnamed: 0,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,04-07-01,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,15-08-00,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [4]:
len(data)

1000

In [5]:
data["abstract"]=data["abstract"].astype(str)

We just care about the text in abstracts now, so we remove all the NaN abstracts

In [6]:
data2 = data[data.abstract != 'nan']


In [7]:
len(data2)

956

In [8]:
#data = pd.read_csv("C:/Users/Iro Sfoungari/Desktop/data_with_lang.csv")


Language detection. There where some symbols in the text, and detect(x) couldn't work! so I had to add an exception 

In [9]:
def lan_detect(x):
    try:
        lang = detect(x)
    except:
        lang = 'Other'
    return lang


In [10]:
tqdm.pandas()
data2["Lang"] = data2["abstract"].progress_apply(lan_detect)

C:\Users\Iro Sfoungari\anacondaDisaster\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|███████████████████████████████████████████████████████████████████████████████| 956/956 [00:06<00:00, 146.27it/s]
C:\Users\Iro Sfoungari\anacondaDisaster\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


store it in a csv locally

In [11]:
#csv_data = data2.to_csv(r'C:/Users/Iro Sfoungari/Desktop/data_with_lang_all.csv', index = False)

drop all rows that contain fr, spanish, ge or other langyages

In [12]:
data3=data2[data2['Lang'].str.contains("en")]

So we have 269223 abstracts to process!

In [13]:
len(data3)

956

In [14]:
data3.head()

,Unnamed: 0,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,...,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,Lang
0,0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,...,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,en
1,1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,...,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,en
2,2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,...,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,en
3,3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,...,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,en
4,4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,...,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,en


<h2> Data preprocessing / Cleaning </h2>

<h1>Spacy tokenization and stopword removal</h1>

In [18]:
def spacy_stropword_rem(sentence):
    mytokens = parser(sentence)
    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

In [19]:
#csv_data = data3.to_csv(r'C:/Users/Iro Sfoungari/Desktop/data_with_lang_onlyenglish_procabstract.csv', index = False)

In [20]:
data  = data3.abstract.tolist()


In [21]:
no_stopwords=list(map(spacy_stropword_rem, data))


In [22]:
print(no_stopwords[1])

['inflammatory', 'diseases', 'respiratory', 'tract', 'commonly', 'associated', 'elevated', 'production', 'nitric', 'oxide', 'no•', 'increased', 'indices', 'no•', '-dependent', 'oxidative', 'stress', 'no•', 'known', 'anti', 'microbial', 'anti', 'inflammatory', 'anti', 'oxidant', 'properties', 'lines', 'evidence', 'support', 'contribution', 'no•', 'lung', 'injury', 'disease', 'models', 'basis', 'biochemical', 'evidence', 'presumed', 'no•', '-dependent', 'oxidations', 'formation', 'oxidant', 'peroxynitrite', 'alternative', 'mechanisms', 'involving', 'phagocyte', 'derived', 'heme', 'proteins', 'myeloperoxidase', 'eosinophil', 'peroxidase', 'operative', 'conditions', 'inflammation', 'overwhelming', 'literature', 'no•', 'generation', 'activities', 'respiratory', 'tract', 'scope', 'commentary', 'review', 'area', 'comprehensively', 'instead', 'focuses', 'recent', 'evidence', 'concepts', 'presumed', 'contribution', 'no•', 'inflammatory', 'diseases', 'lung']


<h2>Clean-up text </h2>


The next step is to tokenize and clean the text. For this I used Gensim’s simple_preprocess() 
Gensim is a Python library for topic modelling, document indexing and similarity retrieval with large corpora.

In [26]:
#some more preprocessing and put text into list 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [27]:
tokenized_clean_text = list(sent_to_words(no_stopwords))



In [29]:
print(tokenized_clean_text[1])

['inflammatory', 'diseases', 'respiratory', 'tract', 'commonly', 'associated', 'elevated', 'production', 'nitric', 'oxide', 'no', 'increased', 'indices', 'no', 'dependent', 'oxidative', 'stress', 'no', 'known', 'anti', 'microbial', 'anti', 'inflammatory', 'anti', 'oxidant', 'properties', 'lines', 'evidence', 'support', 'contribution', 'no', 'lung', 'injury', 'disease', 'models', 'basis', 'biochemical', 'evidence', 'presumed', 'no', 'dependent', 'oxidations', 'formation', 'oxidant', 'peroxynitrite', 'alternative', 'mechanisms', 'involving', 'phagocyte', 'derived', 'heme', 'proteins', 'myeloperoxidase', 'eosinophil', 'peroxidase', 'operative', 'conditions', 'inflammation', 'overwhelming', 'literature', 'no', 'generation', 'activities', 'respiratory', 'tract', 'scope', 'commentary', 'review', 'area', 'comprehensively', 'instead', 'focuses', 'recent', 'evidence', 'concepts', 'presumed', 'contribution', 'no', 'inflammatory', 'diseases', 'lung']


<h2> Bigrams</h2>
Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In [31]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokenized_clean_text, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokenized_clean_text], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[tokenized_clean_text[0]]])


['objective', 'retrospective', 'chart', 'review', 'describes', 'epidemiology', 'clinical', 'features', 'patients', 'culture', 'proven', 'mycoplasma', 'pneumoniae', 'infections', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia']


In [32]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


In [33]:
# Form Bigrams
data_words_bigrams = make_bigrams(tokenized_clean_text)


In [34]:
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [35]:
data_words_trigrams = make_trigrams(data_words_bigrams)


In [36]:
print(data_words_trigrams[1])

['inflammatory', 'diseases', 'respiratory_tract', 'commonly', 'associated', 'elevated', 'production', 'nitric_oxide', 'no', 'increased', 'indices', 'no', 'dependent', 'oxidative_stress', 'no', 'known', 'anti', 'microbial', 'anti_inflammatory', 'anti', 'oxidant', 'properties', 'lines', 'evidence', 'support', 'contribution', 'no', 'lung_injury', 'disease', 'models', 'basis', 'biochemical', 'evidence', 'presumed', 'no', 'dependent', 'oxidations', 'formation', 'oxidant', 'peroxynitrite', 'alternative', 'mechanisms', 'involving', 'phagocyte', 'derived', 'heme', 'proteins', 'myeloperoxidase', 'eosinophil', 'peroxidase', 'operative', 'conditions', 'inflammation', 'overwhelming', 'literature', 'no', 'generation', 'activities', 'respiratory_tract', 'scope', 'commentary', 'review', 'area', 'comprehensively', 'instead', 'focuses', 'recent', 'evidence', 'concepts', 'presumed', 'contribution', 'no', 'inflammatory', 'diseases', 'lung']


<h2>Lemmatization Spacy</h2>

I deactivated allowed_postags, because it was restricting my results 

In [39]:
#spacy lemmatization

def lemmatization(texts):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])
    return texts_out


In [40]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams)


In [44]:
print(data_lemmatized[1])

['inflammatory', 'disease', 'respiratory_tract', 'commonly', 'associate', 'elevated', 'production', 'nitric_oxide', 'no', 'increased', 'indice', 'no', 'dependent', 'oxidative_stress', 'no', 'know', 'anti', 'microbial', 'anti_inflammatory', 'anti', 'oxidant', 'property', 'line', 'evidence', 'support', 'contribution', 'no', 'lung_injury', 'disease', 'model', 'basis', 'biochemical', 'evidence', 'presume', 'no', 'dependent', 'oxidation', 'formation', 'oxidant', 'peroxynitrite', 'alternative', 'mechanism', 'involve', 'phagocyte', 'derive', 'heme', 'proteins', 'myeloperoxidase', 'eosinophil', 'peroxidase', 'operative', 'condition', 'inflammation', 'overwhelm', 'literature', 'no', 'generation', 'activity', 'respiratory_tract', 'scope', 'commentary', 'review', 'area', 'comprehensively', 'instead', 'focus', 'recent', 'evidence', 'concept', 'presume', 'contribution', 'no', 'inflammatory', 'disease', 'lung']
